In [41]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, median_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from scipy.stats import *
from sklearn.preprocessing import RobustScaler

In [42]:
rides = pd.read_csv("../big_query_24.csv")
rides.rename(columns={'id': 'ride_id'}, inplace=True)
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city
count,2.426000e+04,2.426000e+04,24260.000000,24260.000000,24260.000000,24260.000000,24260.000000,24260.000000,2.426000e+04,24260.0
mean,2.010495e+06,2.376807e+07,35.693980,51.401119,35.694668,51.397562,1003.226051,1236.469992,4.956898e+09,1.0
std,1.440427e+06,1.800928e+07,0.096191,0.143753,0.095899,0.152071,725.298062,1034.221594,3.405802e+05,0.0
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.636840,50.016300,24.000000,120.000000,4.956287e+09,1.0
25%,5.573940e+05,5.043194e+06,35.663319,51.331473,35.661003,51.330028,464.000000,504.000000,4.956594e+09,1.0
50%,1.992264e+06,2.400925e+07,35.723982,51.404408,35.723633,51.401817,799.000000,905.000000,4.956982e+09,1.0
75%,3.323714e+06,3.831664e+07,35.755246,51.465694,35.754699,51.470322,1347.000000,1650.000000,4.957167e+09,1.0
max,4.346548e+06,5.750996e+07,35.839420,52.794230,36.259760,52.792877,7794.000000,9088.000000,4.957626e+09,1.0


In [50]:
colnames = ['ride_id','driver_id','a_t_a_result.arrival_a_t_a','a_t_a_result.boarding_a_t_a','a_t_a_result.ride_a_t_a','a_t_a_result.arrival_probe_result.probe.point.lat','a_t_a_result.arrival_probe_result.probe.point.lon','a_t_a_result.arrival_probe_result.probe.timestamp','a_t_a_result.arrival_probe_result.confidence','a_t_a_result.arrival_probe_result.h3_index', 'a_t_a_result.arrival_probe_result.k_ring_level','a_t_a_result.boarding_probe_result.probe.point.lat','a_t_a_result.boarding_probe_result.probe.point.lon','a_t_a_result.boarding_probe_result.probe.timestamp','a_t_a_result.boarding_probe_result.confidence','a_t_a_result.boarding_probe_result.h3_index','a_t_a_result.boarding_probe_result.k_ring_level','a_t_a_result.final_destination_probe_result.probe.point.lat','a_t_a_result.final_destination_probe_result.probe.point.lon','a_t_a_result.final_destination_probe_result.probe.timestamp','a_t_a_result.final_destination_probe_result.confidence','a_t_a_result.final_destination_probe_result.h3_index','a_t_a_result.final_destination_probe_result.k_ring_level','a_t_a_result.destination_probe_result.probe.point.lat','a_t_a_result.destination_probe_result.probe.point.lon','a_t_a_result.destination_probe_result.probe.timestamp','a_t_a_result.destination_probe_result.confidence','a_t_a_result.destination_probe_result.h3_index','a_t_a_result.destination_probe_result.k_ring_level','a_t_a_result.extra_destination_probe_result.probe.point.lat','a_t_a_result.extra_destination_probe_result.probe.point.lon','a_t_a_result.extra_destination_probe_result.probe.timestamp','a_t_a_result.extra_destination_probe_result.confidence','a_t_a_result.extra_destination_probe_result.h3_index','a_t_a_result.extra_destination_probe_result.k_ring_level','pickup_a_d_d_result.distance','pickup_a_d_d_result.confidence','pickup_a_d_d_result.route_ratio','pickup_a_d_d_result.g_p_s_ratio','ride_a_d_d_result.distance','ride_a_d_d_result.confidence','ride_a_d_d_result.route_ratio','ride_a_d_d_result.g_p_s_ratio','total_a_d_d_confidence','in_ride_allotment','e_d_d','clickhouse_time','hash']
khatkesh = pd.read_csv("../khatkesh.csv", names=colnames)
khatkesh.describe()

,ride_id,driver_id,a_t_a_result.arrival_a_t_a,a_t_a_result.boarding_a_t_a,a_t_a_result.ride_a_t_a,a_t_a_result.arrival_probe_result.probe.point.lat,a_t_a_result.arrival_probe_result.probe.point.lon,a_t_a_result.arrival_probe_result.probe.timestamp,a_t_a_result.arrival_probe_result.confidence,a_t_a_result.arrival_probe_result.k_ring_level,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,5.028590e+05,5.028590e+05,502859.000000,5.028590e+05,5.028590e+05,502859.000000,502859.000000,5.028590e+05,502859.000000,502859.000000,...,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,5.028590e+05
mean,4.956607e+09,2.379701e+06,186.070441,2.013937e+16,7.155686e+07,34.603114,51.913875,2.457812e+15,0.734051,1.158275,...,0.000326,0.015392,13.564964,0.857177,0.000125,0.707576,0.867706,0.046938,8.377787,9.240882e+18
std,4.392595e+05,1.344870e+06,146.035298,6.091805e+17,2.135430e+09,2.544448,3.538870,2.129146e+17,0.341657,1.439362,...,0.015817,0.017221,17.250296,0.189048,0.005345,0.344261,0.178898,0.332769,12.547627,5.326126e+18
min,4.873587e+09,1.000000e+00,0.000000,0.000000e+00,1.000000e+00,25.277332,44.829102,1.662865e+09,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.062941e+13
25%,4.956373e+09,1.199125e+06,91.000000,8.000000e+00,4.020000e+02,32.695423,50.836760,1.663160e+09,0.591716,0.000000,...,0.000000,0.005714,4.603000,0.750032,0.000000,0.482860,0.792329,0.000000,2.784000,4.635700e+18
50%,4.956612e+09,2.438832e+06,170.000000,3.600000e+01,6.840000e+02,35.707092,51.409603,1.663161e+09,0.769231,1.000000,...,0.000000,0.011905,8.623000,0.957293,0.000000,0.884615,0.957387,0.000000,5.180000,9.246887e+18
75%,4.956863e+09,3.616538e+06,258.000000,1.240000e+02,1.172000e+03,36.264671,52.524117,1.663163e+09,1.000000,2.000000,...,0.000000,0.020833,16.327728,1.000000,0.000000,0.975610,1.000000,0.000000,9.833000,1.385216e+19
max,4.957263e+09,4.346386e+06,7954.000000,1.844674e+19,6.379876e+10,39.661167,61.168961,1.844674e+19,1.000000,4.000000,...,1.000000,1.000000,1784.723999,1.000000,1.000000,1.000000,1.000000,34.022999,1741.516968,1.844670e+19


,ride_id,driver_id,a_t_a_result.arrival_a_t_a,a_t_a_result.boarding_a_t_a,a_t_a_result.ride_a_t_a,a_t_a_result.arrival_probe_result.probe.point.lat,a_t_a_result.arrival_probe_result.probe.point.lon,a_t_a_result.arrival_probe_result.probe.timestamp,a_t_a_result.arrival_probe_result.confidence,a_t_a_result.arrival_probe_result.k_ring_level,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,5.028590e+05,5.028590e+05,502859.000000,5.028590e+05,5.028590e+05,502859.000000,502859.000000,5.028590e+05,502859.000000,502859.000000,...,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,5.028590e+05
mean,4.956607e+09,2.379701e+06,186.070441,2.013937e+16,7.155686e+07,34.603114,51.913875,2.457812e+15,0.734051,1.158275,...,0.000326,0.015392,13.564964,0.857177,0.000125,0.707576,0.867706,0.046938,8.377787,9.240882e+18
std,4.392595e+05,1.344870e+06,146.035298,6.091805e+17,2.135430e+09,2.544448,3.538870,2.129146e+17,0.341657,1.439362,...,0.015817,0.017221,17.250296,0.189048,0.005345,0.344261,0.178898,0.332769,12.547627,5.326126e+18
min,4.873587e+09,1.000000e+00,0.000000,0.000000e+00,1.000000e+00,25.277332,44.829102,1.662865e+09,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.062941e+13
25%,4.956373e+09,1.199125e+06,91.000000,8.000000e+00,4.020000e+02,32.695423,50.836760,1.663160e+09,0.591716,0.000000,...,0.000000,0.005714,4.603000,0.750032,0.000000,0.482860,0.792329,0.000000,2.784000,4.635700e+18
50%,4.956612e+09,2.438832e+06,170.000000,3.600000e+01,6.840000e+02,35.707092,51.409603,1.663161e+09,0.769231,1.000000,...,0.000000,0.011905,8.623000,0.957293,0.000000,0.884615,0.957387,0.000000,5.180000,9.246887e+18
75%,4.956863e+09,3.616538e+06,258.000000,1.240000e+02,1.172000e+03,36.264671,52.524117,1.663163e+09,1.000000,2.000000,...,0.000000,0.020833,16.327728,1.000000,0.000000,0.975610,1.000000,0.000000,9.833000,1.385216e+19
max,4.957263e+09,4.346386e+06,7954.000000,1.844674e+19,6.379876e+10,39.661167,61.168961,1.844674e+19,1.000000,4.000000,...,1.000000,1.000000,1784.723999,1.000000,1.000000,1.000000,1.000000,34.022999,1741.516968,1.844670e+19


,ride_id,driver_id,a_t_a_result.arrival_a_t_a,a_t_a_result.boarding_a_t_a,a_t_a_result.ride_a_t_a,a_t_a_result.arrival_probe_result.probe.point.lat,a_t_a_result.arrival_probe_result.probe.point.lon,a_t_a_result.arrival_probe_result.probe.timestamp,a_t_a_result.arrival_probe_result.confidence,a_t_a_result.arrival_probe_result.k_ring_level,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,5.028590e+05,5.028590e+05,502859.000000,5.028590e+05,5.028590e+05,502859.000000,502859.000000,5.028590e+05,502859.000000,502859.000000,...,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,5.028590e+05
mean,4.956607e+09,2.379701e+06,186.070441,2.013937e+16,7.155686e+07,34.603114,51.913875,2.457812e+15,0.734051,1.158275,...,0.000326,0.015392,13.564964,0.857177,0.000125,0.707576,0.867706,0.046938,8.377787,9.240882e+18
std,4.392595e+05,1.344870e+06,146.035298,6.091805e+17,2.135430e+09,2.544448,3.538870,2.129146e+17,0.341657,1.439362,...,0.015817,0.017221,17.250296,0.189048,0.005345,0.344261,0.178898,0.332769,12.547627,5.326126e+18
min,4.873587e+09,1.000000e+00,0.000000,0.000000e+00,1.000000e+00,25.277332,44.829102,1.662865e+09,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.062941e+13
25%,4.956373e+09,1.199125e+06,91.000000,8.000000e+00,4.020000e+02,32.695423,50.836760,1.663160e+09,0.591716,0.000000,...,0.000000,0.005714,4.603000,0.750032,0.000000,0.482860,0.792329,0.000000,2.784000,4.635700e+18
50%,4.956612e+09,2.438832e+06,170.000000,3.600000e+01,6.840000e+02,35.707092,51.409603,1.663161e+09,0.769231,1.000000,...,0.000000,0.011905,8.623000,0.957293,0.000000,0.884615,0.957387,0.000000,5.180000,9.246887e+18
75%,4.956863e+09,3.616538e+06,258.000000,1.240000e+02,1.172000e+03,36.264671,52.524117,1.663163e+09,1.000000,2.000000,...,0.000000,0.020833,16.327728,1.000000,0.000000,0.975610,1.000000,0.000000,9.833000,1.385216e+19
max,4.957263e+09,4.346386e+06,7954.000000,1.844674e+19,6.379876e+10,39.661167,61.168961,1.844674e+19,1.000000,4.000000,...,1.000000,1.000000,1784.723999,1.000000,1.000000,1.000000,1.000000,34.022999,1741.516968,1.844670e+19


,ride_id,driver_id,a_t_a_result.arrival_a_t_a,a_t_a_result.boarding_a_t_a,a_t_a_result.ride_a_t_a,a_t_a_result.arrival_probe_result.probe.point.lat,a_t_a_result.arrival_probe_result.probe.point.lon,a_t_a_result.arrival_probe_result.probe.timestamp,a_t_a_result.arrival_probe_result.confidence,a_t_a_result.arrival_probe_result.k_ring_level,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,5.028590e+05,5.028590e+05,502859.000000,5.028590e+05,5.028590e+05,502859.000000,502859.000000,5.028590e+05,502859.000000,502859.000000,...,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,5.028590e+05
mean,4.956607e+09,2.379701e+06,186.070441,2.013937e+16,7.155686e+07,34.603114,51.913875,2.457812e+15,0.734051,1.158275,...,0.000326,0.015392,13.564964,0.857177,0.000125,0.707576,0.867706,0.046938,8.377787,9.240882e+18
std,4.392595e+05,1.344870e+06,146.035298,6.091805e+17,2.135430e+09,2.544448,3.538870,2.129146e+17,0.341657,1.439362,...,0.015817,0.017221,17.250296,0.189048,0.005345,0.344261,0.178898,0.332769,12.547627,5.326126e+18
min,4.873587e+09,1.000000e+00,0.000000,0.000000e+00,1.000000e+00,25.277332,44.829102,1.662865e+09,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.062941e+13
25%,4.956373e+09,1.199125e+06,91.000000,8.000000e+00,4.020000e+02,32.695423,50.836760,1.663160e+09,0.591716,0.000000,...,0.000000,0.005714,4.603000,0.750032,0.000000,0.482860,0.792329,0.000000,2.784000,4.635700e+18
50%,4.956612e+09,2.438832e+06,170.000000,3.600000e+01,6.840000e+02,35.707092,51.409603,1.663161e+09,0.769231,1.000000,...,0.000000,0.011905,8.623000,0.957293,0.000000,0.884615,0.957387,0.000000,5.180000,9.246887e+18
75%,4.956863e+09,3.616538e+06,258.000000,1.240000e+02,1.172000e+03,36.264671,52.524117,1.663163e+09,1.000000,2.000000,...,0.000000,0.020833,16.327728,1.000000,0.000000,0.975610,1.000000,0.000000,9.833000,1.385216e+19
max,4.957263e+09,4.346386e+06,7954.000000,1.844674e+19,6.379876e+10,39.661167,61.168961,1.844674e+19,1.000000,4.000000,...,1.000000,1.000000,1784.723999,1.000000,1.000000,1.000000,1.000000,34.022999,1741.516968,1.844670e+19


In [51]:
len(colnames)

48

In [52]:
rides_khatkesh = pd.merge(rides, khatkesh, on="ride_id")
rides_khatkesh.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,1.506300e+04,1.506300e+04,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,1.506300e+04,15063.0,...,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,1.506300e+04
mean,1.981823e+06,2.391234e+07,35.693123,51.402332,35.694000,51.399399,867.554870,1088.631083,4.956710e+09,1.0,...,0.000208,0.014827,13.534466,0.876051,0.000115,0.738491,0.883905,0.015693,8.121885,9.220426e+18
std,1.436073e+06,1.795970e+07,0.097267,0.146990,0.097062,0.152614,633.272365,951.193802,2.752267e+05,0.0,...,0.011839,0.015742,14.004605,0.174419,0.002896,0.321243,0.164876,0.146355,8.625966,5.339763e+18
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.641853,50.535954,24.000000,120.000000,4.956287e+09,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.232000,3.680090e+14
25%,5.346035e+05,5.211465e+06,35.662340,51.330618,35.660725,51.330080,410.000000,436.000000,4.956454e+09,1.0,...,0.000000,0.005319,4.448000,0.813760,0.000000,0.582005,0.835089,0.000000,2.646000,4.576592e+18
50%,1.940900e+06,2.427381e+07,35.724340,51.404480,35.724358,51.402800,689.000000,775.000000,4.956690e+09,1.0,...,0.000000,0.011236,8.968000,0.965429,0.000000,0.900990,0.963664,0.000000,5.365000,9.171676e+18
75%,3.289352e+06,3.849748e+07,35.754384,51.468884,35.754307,51.472001,1138.000000,1392.000000,4.956964e+09,1.0,...,0.000000,0.020000,17.646500,1.000000,0.000000,0.974576,1.000000,0.000000,10.529500,1.387260e+19
max,4.346206e+06,5.750970e+07,35.832066,52.781403,36.048410,52.792877,5231.000000,6709.000000,4.957246e+09,1.0,...,1.000000,0.500000,159.207718,1.000000,0.204274,1.000000,1.000000,4.430000,116.138000,1.844592e+19


In [24]:
khatkesh[khatkesh['ride_id']== 4957183665]

,ride_id,driver_id,a_t_a_result.arrival_a_t_a,a_t_a_result.boarding_a_t_a,a_t_a_result.ride_a_t_a,a_t_a_result.arrival_probe_result.probe.point.lat,a_t_a_result.arrival_probe_result.probe.point.lon,a_t_a_result.arrival_probe_result.probe.timestamp,a_t_a_result.arrival_probe_result.confidence,a_t_a_result.arrival_probe_result.h3_index,...,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,clickhouse_time,hash


In [9]:
rides = rides_khatkesh[(rides_khatkesh['eta'] > 180) &
                       (rides_khatkesh['eta'] < 10800) &
                       (rides_khatkesh['a_t_a_result.ride_a_t_a'] > 180) &
                       (rides_khatkesh['a_t_a_result.ride_a_t_a'] < 10800)]
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
rides = rides[rides['ride_a_d_d_result.confidence'] > 0.6]
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
rides = rides[['eta', 'ata', 'a_t_a_result.ride_a_t_a', 'source_lat', 'source_lng', 'destination_lat', 'destination_lng', 'e_d_d']]
eta_edd = rides[["eta", "e_d_d"]].to_numpy().reshape(-1, 2)
ata = rides["a_t_a_result.ride_a_t_a"].to_numpy().reshape(-1, 1)

In [13]:
ata

array([], shape=(0, 1), dtype=float64)

In [12]:
robust_ata = RobustScaler().fit(ata)
robust_ata_transformed = robust_ata.transform(ata)

robust_ata_df = pd.DataFrame(robust_ata_transformed, columns = ['ATA'])
robust_ata_df.hist(bins=500)

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by RobustScaler.

In [ ]:
eta = eta_edd[:, 0].reshape(-1, 1)
robust_eta = RobustScaler().fit_transform(eta)

robust_eta_df = pd.DataFrame(robust_eta, columns=['robust_eta'])
robust_eta_df.hist(bins=500)

In [ ]:
edd = eta_edd[:, 1].reshape(-1, 1)
robust_edd = RobustScaler().fit_transform(edd)

robust_edd_df = pd.DataFrame(robust_edd, columns=['robust_edd'])
robust_edd_df.hist(bins=500)

In [ ]:
robust_eta_edd = RobustScaler().fit_transform(eta_edd)

robust_eta_edd_df = pd.DataFrame(robust_eta_edd, columns=['robust_eta_edd'])
robust_eta_edd_df.hist(bins=500)

In [6]:
!head ../khatkesh.csv

4956357421,2303267,0,471,3483,35.75701904296875,51.15105056762695,1663159629,0.0,8c2cf3c4c24b3ff,4,35.75701904296875,51.15105056762695,1663160100,0.0,8c2cf3c4c24b3ff,4,35.71916961669922,51.35239028930664,1663164183,0.0,8c2cf3032cf2bff,4,35.71916961669922,51.35239028930664,1663164183,0.0,8c2cf3032cf2bff,4,0.0,0.0,0,0.0,,0,0.0,0.0,0.0,0.0,27.2189998626709,0.5,0.0,0.0,0.44999998807907104,0.0,22.547000885009766,2022-09-14 18:34:47,8019372149383439335
4956939497,2303267,0,572,1618,35.7213134765625,51.34894561767578,1663163739,0.0,8c2cf3032c223ff,4,35.7213134765625,51.34894561767578,1663164311,0.0,8c2cf3032c223ff,4,35.68087387084961,51.25920867919922,1663166529,0.0,8c2cf31843763ff,4,35.68087387084961,51.25920867919922,1663166529,0.0,8c2cf31843763ff,4,0.0,0.0,0,0.0,,0,0.0,0.0,0.0,0.0,14.144000053405762,0.5,0.0,0.0,0.44999998807907104,0.0,16.09000015258789,2022-09-14 19:47:30,17363237605350224929
4956061147,2303290,72,24,1648,29.65280532836914,52.48612594604492,1663156847,1.0,8c433183434c9ff,0